In [26]:
# 1. 
from math import ceil


N = 1024
grids = 8
threadsInBlock = 128

warpsPerBlock = ceil(128 // 32)
print(f"a. {warpsPerBlock} warpsPerBlock")
print(f"b. {grids * warpsPerBlock} warpsInGrid")
print(f"c.1 {3 * grids} Active Warps in Grid, in Line 04 divergence")
print(f"c.2 {2 * grids} Warps divergent in grid, in Line 04 divergence")
print(f"c.3 {32/32*100}% SIMD efficiency warp 0 of block 0 , in Line 04 divergence")
print(f"c.4 {(32-(64-40))/32*100}% SIMD efficiency warp 1 of block 0 , in Line 04 divergence")
print(f"c.4 {(128-104)/32*100}% SIMD efficiency warp 3 of block 0 , in Line 04 divergence")

print(f"d.1 {grids * warpsPerBlock} Warps in grid Active, Line 07 divergence")
print(f"d.2 {grids * warpsPerBlock } Warps in grid Divergent, Line 07 divergence")
print(f"d.2 {50/100*100}% SIMD efficiency warp 0 block 0, Line 07 divergence")
# Loop condition: j: 0 to < 5 - i%3
print(f"e.1 {3} Itrations have NO divergence, Line 09 divergence")
print(f"e.1 {2} Itrations HAVE divergence, Line 09 divergence")


a. 4 warpsPerBlock
b. 32 warpsInGrid
c.1 24 Active Warps in Grid, in Line 04 divergence
c.2 16 Warps divergent in grid, in Line 04 divergence
c.3 100.0% SIMD efficiency warp 0 of block 0 , in Line 04 divergence
c.4 25.0% SIMD efficiency warp 1 of block 0 , in Line 04 divergence
c.4 75.0% SIMD efficiency warp 3 of block 0 , in Line 04 divergence
d.1 32 Warps in grid Active, Line 07 divergence
d.2 32 Warps in grid Divergent, Line 07 divergence
d.2 50.0% SIMD efficiency warp 0 block 0, Line 07 divergence
e.1 3 Itrations have NO divergence, Line 09 divergence
e.1 2 Itrations HAVE divergence, Line 09 divergence
